In [22]:
import pymor, ngsolve
print(pymor.__version__, ngsolve.__version__)


2024.2.0 6.2.2505


In [20]:
# ngsolve + pymor example: Poisson problem
#
# Requires:
#   pip install ngsolve pymor

from ngsolve import *
from netgen.geom2d import unit_square

from pymor.bindings.ngsolve import (
    NGSolveMatrixOperator,
    NGSolveVectorSpace
)

# --------------------------------------------------------------------
# Step 1. Build an NGSolve model: Poisson equation on unit square
# --------------------------------------------------------------------
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

V = H1(mesh, order=2, dirichlet=".*")   # H1 space with Dirichlet BCs

u, v = V.TnT()

a = BilinearForm(grad(u) * grad(v) * dx).Assemble()
f = LinearForm(v * dx).Assemble()

# --------------------------------------------------------------------
# Step 2. Wrap NGSolve objects as pyMOR operators/vectors
# --------------------------------------------------------------------
space = NGSolveVectorSpace(V)
op = NGSolveMatrixOperator(a.mat, source=space, range=space)
rhs = space.make_array([f.vec])

# --------------------------------------------------------------------
# Step 3. Solve via pyMOR operator interface
# --------------------------------------------------------------------
u_sol = op.apply_inverse(rhs)   # still a pyMOR VectorArray

# --------------------------------------------------------------------
# Step 4. Export solution back to NGSolve for visualization
# --------------------------------------------------------------------
u_ngsolve = GridFunction(V)
# take the first vector from the VectorArray, get the NGSolve BaseVector
u_ngsolve.vec.data = u_sol[0].as_vector().data

Draw(u_ngsolve)
print("||u||_L2 =", u_sol.norm())


AttributeError: 'ngsolve.la.BaseVector' object has no attribute 'vec'